# 微分可能LUTモデルによるMNIST学習のHLSサンプル

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。
ここでは HLS に出力することを目的にシンプルな多層パーセプトロンモデルを作成します。

## 事前準備

In [1]:
import os
import shutil
import numpy as np
#from tqdm.notebook import tqdm
from tqdm import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

/home/ryuji/git-work/BinaryBrain/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#bb.set_host_only(True)
print(bb.get_device_name(0))
bb.set_device(0)

NVIDIA GeForce GTX 1660 SUPER


異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [3]:
# configuration
data_path             = './data/'
net_name              = 'MnistDifferentiableLutHls'
data_path             = os.path.join('./data/', net_name)
hls_function_name     = 'MnistLut'
hls_output_file       = os.path.join(data_path, net_name + '.h')
hls_src_path          = '../../hls/mnist/simple/src'
hls_src_file          = os.path.join(hls_src_path, net_name + '.h')

epochs                = 8
mini_batch_size       = 64

In [4]:
os.makedirs(data_path, exist_ok=True)
os.makedirs(hls_src_path, exist_ok=True)

データセットは PyTorch の torchvision を使います。ミニバッチのサイズも DataLoader で指定しています。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [5]:
# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

接続数がLUTの物理構成に合わせて、1ノード当たり6個なので層間で6倍以上ノード数が違うと接続されないノードが発生するので、注意してネットワーク設計が必要です。
最終段は各クラス7個の結果を出して Reduce で足し合わせています。こうすることで若干の改善がみられるとともに、加算結果が INT3 相当になるために若干尤度を数値的に見ることができるようです。

In [6]:
# define network
net = bb.Sequential([
            bb.Binarize(binary_th=0.5, binary_low=0.0, binary_high=1.0),
            bb.DifferentiableLut([512]),
            bb.DifferentiableLut([256]),
            bb.DifferentiableLut([10, 64]),
            bb.DepthwiseDenseAffineQuantize([10])
        ])

net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam(learning_rate=0.0001)

net.print_info()

----------------------------------------------------------------------
[Sequential] 
 input  shape : [1, 28, 28] output shape : [10]
  --------------------------------------------------------------------
  [Binarize] 
   input  shape : {1, 28, 28} output shape : {1, 28, 28}
  --------------------------------------------------------------------
  [DifferentiableLut6] 
   input  shape : {1, 28, 28} output shape : {512}
   binary : 1   batch_norm : 1
  --------------------------------------------------------------------
  [DifferentiableLut6] 
   input  shape : {512} output shape : {256}
   binary : 1   batch_norm : 1
  --------------------------------------------------------------------
  [DifferentiableLut6] 
   input  shape : {256} output shape : {10, 64}
   binary : 1   batch_norm : 1
  --------------------------------------------------------------------
  [DepthwiseDenseAffine] 
   input  shape : {10, 64} output shape : {10}
   input(64, 10) output(1, 10)
----------------------------

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [7]:
#bb.load_networks(data_path, net)

# learning
optimizer.set_variables(net.get_parameters(), net.get_gradients())
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)

            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)

            net.backward(dy_buf)

            optimizer.update()
        
            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)
    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))
    
    bb.save_networks(data_path, net)

100%|████████████████████████████████████████████████████████████| 938/938 [00:03<00:00, 276.22it/s, acc=0.8, loss=1.28]


epoch[0] : loss=0.793833 accuracy=0.871300


100%|█████████████████████████████████████████████████████████| 938/938 [00:03<00:00, 261.13it/s, acc=0.897, loss=0.573]


epoch[1] : loss=0.515941 accuracy=0.900300


100%|█████████████████████████████████████████████████████████| 938/938 [00:03<00:00, 262.18it/s, acc=0.916, loss=0.398]


...

epoch[5] : loss=0.286051 accuracy=0.928000


100%|█████████████████████████████████████████████████████████| 938/938 [00:03<00:00, 257.94it/s, acc=0.938, loss=0.235]


epoch[6] : loss=0.256886 accuracy=0.931000


100%|██████████████████████████████████████████████████████████| 938/938 [00:03<00:00, 266.92it/s, acc=0.941, loss=0.22]


epoch[7] : loss=0.265055 accuracy=0.924900


## FPGA用HLS(C言語高位合成)で使う為の出力

内部データを取得する例としてHSL(C言語高位合成)用の出力を作ってみます

In [8]:
# 学習済みを読みなおす
bb.load_networks(data_path, net)

# HLSソースを出力
with open(hls_output_file, "w") as f:
    # header
    f.write('// BinaryBrain MnistDifferentiableLut HLS sample\n\n')
    f.write('#include "ap_int.h"\n\n')
    
    # LUT-Net 出力
    for i in range(1, 4):
        bb.dump_hls_lut_layer(f, hls_function_name + "_layer%d"%i, net[i])
    f.write('\n\n')
    
    # DenseAffine parameter
    W = (net[4].WQ().numpy() * 256).astype(np.int32)
    b = (net[4].bQ().numpy() * 256).astype(np.int32)
    f.write('const int DWA_DEPTH = %d;\n'%W.shape[2])
    f.write('const ap_int<8> W_tbl[%d][DWA_DEPTH] =\n'%(W.shape[0]))
    f.write('    {\n')
    for i in range(W.shape[0]):
        f.write('        {')
        for j in range(W.shape[2]):
            f.write('%5d, '%W[i][0][j])
        f.write('        },\n')
    f.write('    };\n\n')
    
    f.write('const ap_int<8> b_tbl[DWA_DEPTH] = {')
    for i in range(b.shape[0]):
        f.write('%5d, '%b[i])
    f.write('};\n\n')

In [9]:
# Simulation用ファイルに上書きコピー
shutil.copyfile(hls_output_file, hls_src_file)

'../../hls/mnist/simple/src/MnistDifferentiableLutHls.h'